### README - zipcodes_get_geos.ipynb 

This notebook will extract the zipcodes from ../downloads/zip_codes/zipcodes.txt and produce a YAML file containing the ZIP Code along with the lat/lon geo-coordinates for the ZIP Code. This YAML file will be named "zipcodes_city_state_.yml" where city and state values are taken from the config.yml file in ../source. If the zipcodes.txt file is present per above and the location_info section of the config.yml is as desired, this notebook should run as is.

**Note:** please be sure to update the city and state names in the [config.yml](../source/config.yml) file prior to executing this notebook. 

In [1]:
import json
import os
import sqlite3
import sys
from datetime import datetime

import logzero
import numpy as np
import pandas as pd
import yaml
from logzero import logger

In [2]:
sys.path.append("../source")
import queries

In [3]:
log_path = "logs/"
log_file = "zipcode_geos.log"

logzero.logfile(log_path + log_file, maxBytes=1e5, backupCount=5, disableStderrLogger=True)

logger.info(f"{log_path}, {log_file}")
logger.info(sys.path)

In [4]:
try:
    with open("../source/config.yml", "r") as config_in:
        configs = yaml.load(config_in, Loader=yaml.SafeLoader)
        logger.info(configs)
except:
    logger.error(f"config file open failure.")
    exit(1)

data_path = configs["file_paths"]["downloads_path_zips"]
data_file = configs["file_names"]["data_zipcodes"]

db_path = configs["file_paths"]["db_path"]
db_file = configs["file_names"]["db_file_gzc"]

city = configs["location_info"]["city"]
state = configs["location_info"]["state"]
loc_name = city + "_" + state

In [5]:
logger.info(f"{data_path}, {data_file}")
logger.info(f"{db_path}, {db_file}")
logger.info(f"{city}, {state}")

In [6]:
zip_list = []

try:
    with open(data_path + data_file, "r") as fh:
        lines = fh.readlines()
        for line in lines:
            zip_list.append(line.strip())
except:
    print(
        f"File read error. Is the file zipcodes.txt present in {configs['file_paths']['downloads_path_zips']}"
    )
    logger.info(
        f"File read error. Is the file zipcodes.txt present in {configs['file_paths']['downloads_path_zips']}"
    )


logger.info(f"{zip_list}")

In [7]:
# establish db connection and cursor
conn = sqlite3.connect(db_path + db_file)
cursor = conn.cursor()

In [8]:
zipcodes = {}

for zip in zip_list:
    cursor.execute(queries.select_zipcode_geo, {"zipcode": zip})
    zip_data = cursor.fetchall()
    zipcodes[zip_data[0][0]] = {"lat": zip_data[0][1], "lon": zip_data[0][2]}

logger.info(f"{zipcodes}\n")

In [9]:
try:
    with open(data_path + "zipcodes_" + loc_name + ".yml", "w") as yml_out:
        yaml.dump({"zipcodes": zipcodes}, yml_out)
        logger.info(f"yaml dump successful\n")
        os.rename(data_path + data_file, data_path + "zipcodes_" + loc_name + ".txt")
        logger.info(f"zip_codes.txt renamed to {'zipcodes_' + loc_name + '.txt'}")
except:
    logger.error("yaml file open error")

In [10]:
conn.close()